# Prediction Using Random Forest

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Importing the Necessary Libraries<br></p>

In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

## Plot Includes

In [2]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

<p style="font-family: Arial; font-size:1.75em;color:purple; font-style:bold"><br>

Creating a Pandas DataFrame from a CSV file<br></p>

In [3]:
def CreateDataFrame(types, level):
    data = pd.read_csv('../Extraction/'+types+"/"+level+'.csv')
    data["Class"] = level
    return data

In [4]:
def LoadData(types):
    frames = [CreateDataFrame(types,"Clean"), 
          CreateDataFrame(types,"Dirty (0-20)"), 
          CreateDataFrame(types,"Dirty (20-40)"), 
          CreateDataFrame(types,"Dirty (40-60)"),
          CreateDataFrame(types,"Dirty (60-80)"),
          CreateDataFrame(types,"Dirty (80-100)")]
    df = pd.concat(frames)
    y=df[['Class']].copy()
    X = df.copy()
    del X['Class']
    return X,y

# Classification using Random Forest

## Optimize Classfiers

In [56]:
def optimize(X_train, y_train):
    param_grid = [
            {'n_estimators': [25,30],'max_features':['auto']},
            ]

    clf = RandomForestClassifier()
    grid_search = GridSearchCV(clf, param_grid, cv=5, verbose=3, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())
    return grid_search

## Cycle Work

In [57]:
def cycleWork(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    gridsearch = optimize(X_train, y_train)
    print(gridsearch.best_params_)
    estimator = gridsearch.best_estimator_
    
    for i in range (0,len(estimator.feature_importances_)):
        if(estimator.feature_importances_[i]>0.04):
            print(X.columns[i]," - ",estimator.feature_importances_[i])
    y_train_pred = estimator.predict(X_train)
    trnscore = accuracy_score(y_train, y_train_pred)
    
    y_tst_pred = estimator.predict(X_test)
    tstscore = accuracy_score(y_test, y_tst_pred)
    cm = confusion_matrix(y_test, y_tst_pred)
    true_pos = np.diag(cm) 
    precision = np.sum(true_pos / np.sum(cm, axis=0))
    recall = np.sum(true_pos / np.sum(cm, axis=1))
    f1 = 2 * ((precision* recall)/(precision + recall))
    #f1None = f1_score(y_test, y_tst_pred, average=None)
    #f1micro = f1_score(y_test, y_tst_pred, average='micro')
    return [trnscore, tstscore, f1]

In [58]:
def AllWork(types,cycles = 5):
    data = []
    X,y = LoadData(types)
    for i in range(0,cycles):
        print(i)
        acc = cycleWork(X,y)
        print("Train Accuracy ",acc[0],", Test Accuracy",acc[1], "F1 ",acc[2])
        data.append([acc[0],acc[1],acc[2]])
        numpyarray = np.array(data)
        df = pd.DataFrame(numpyarray, columns = ["Train Acc", "Test Acc", "F1 Score"])
        df.to_csv("RandomForest-"+types+'.csv',index = False)

In [59]:
types = "Solar Data"
AllWork(types,5)
types = "NoBG"
AllWork(types,5)

0
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.9s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.4s finished


{'max_features': 'auto', 'n_estimators': 30}
GSkew  -  0.0465360007992
BKurt  -  0.0477431320994
GrayKurt  -  0.0407394093326
BlackRatio  -  0.0560891961743
Train Accuracy  1.0 , Test Accuracy 0.967684021544 F1  5.83675044243
1
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   11.8s remaining:    5.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.3s finished


{'max_features': 'auto', 'n_estimators': 30}
BKurt  -  0.0433948429789
GraySkew  -  0.0448259545918
BlackRatio  -  0.0443202470931
WhiteRatio  -  0.0542544711641
Train Accuracy  1.0 , Test Accuracy 0.96947935368 F1  5.83316222596
2
Fitting 5 folds for each of 2 candidates, totalling 10 fits


...

{'max_features': 'auto', 'n_estimators': 30}
ASkew  -  0.0406448287751
RSkew  -  0.0928266141029
RKurt  -  0.0594550852969
GSkew  -  0.0805515958995
GKurt  -  0.0426268109068
GraySkew  -  0.0669485194555
GrayKurt  -  0.0414598934088
Correlation90  -  0.0477346947764
Train Accuracy  1.0 , Test Accuracy 0.94973070018 F1  5.68161743825
3
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    6.4s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    6.9s finished


{'max_features': 'auto', 'n_estimators': 25}
ASkew  -  0.0613536849665
RSkew  -  0.0843634209965
RKurt  -  0.0742612397423
GSkew  -  0.0688713091508
GraySkew  -  0.0615914463952
Train Accuracy  1.0 , Test Accuracy 0.964093357271 F1  5.79260552718
4
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.7s remaining:    4.1s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   10.2s finished


{'max_features': 'auto', 'n_estimators': 30}
RSkew  -  0.0988841489768
RKurt  -  0.0470790441089
GSkew  -  0.0894301085569
GKurt  -  0.0510089064923
GraySkew  -  0.0625525270934
GrayKurt  -  0.0405497640211
Correlation45  -  0.0427414047513
Correlation90  -  0.0400236595002
Train Accuracy  1.0 , Test Accuracy 0.960502692998 F1  5.7825221356
